<a href="https://colab.research.google.com/github/franciscogarate/cdiae/blob/main/notebooks/4_limpieza_outliers_California.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza de *outliers* o valores atípicos

La limpieza de *outliers* o valores atípicos es un proceso importante en el análisis de datos. Los *outliers* son valores que se desvían significativamente de los valores esperados o normales. Estos valores pueden ser el resultado de errores de medición, errores de transcripción, o simplemente valores que no son representativos de la población.

Aunque se pierdan registros, es importante eliminar dichos *outliers* para obtener resultados más precisos.

Muchas veces, el verdadero éxito de una estimación con modelos predictivos o *machine learning* es la tarea previa de limpieza de valores atípicos.

In [ ]:
!git clone https://github.com/franciscogarate/cdiae

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from scipy import stats

Aprovechamos que la propia librería scikit-learn dispone de datasets para su uso sin necesidad de importar ficheros. A tal efecto, cargamos el dataset de California Housing desde scikit-learn:

In [ ]:
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing.target                                           # Añadir variable objetivo (precio medio de la vivienda)
df.head()

Para conocer lo que representa cada columna, mostramos la descripción textual del dataset

In [ ]:
print(housing.DESCR)

In [ ]:
df.info()

Estadísticas descriptivas iniciales: Realizamos un análisis exploratorio inicial con las principales estadísticas descriptivas, fijandonos especialmente en los valores máximos (por ejemplo, viviendas con 34 dormitorios cuando la la media es de 1.09 dormitorios por vivienda)

In [ ]:
df.describe()

Creamos una función para visualizar outliers columna a columna con boxplots (librería matplotlib)

In [ ]:
def plot_outliers(df, title=""):
    fig, axes = plt.subplots(3, 3, figsize=(12, 9))    # Crear rejilla 3x3 de subgráficos
    fig.suptitle(title, fontsize=16)

    columns = df.columns
    for i, col in enumerate(columns):
        row = i // 3            # Calcular posición en la rejilla
        col_idx = i % 3
        axes[row, col_idx].boxplot(df[col])
        axes[row, col_idx].set_title(col)
        axes[row, col_idx].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

Visualizamos con la función anterior los outliers iniciales (antes de limpiar)

In [ ]:
plot_outliers(df, 'Outliers antes de la Limpieza')

Definimos las funciones de limpieza:
- `detectar_outliers_iqr`: Detecta outliers usando el método `IQR` (rangos intercuartílicos)
- `detectar_outliers_zscore`: Detecta outliers usando `Z-score` (máscara por umbrales)

In [ ]:
def detectar_outliers_iqr(series, multiplier=1.5):
    # Cuartiles y rango intercuartílico
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    # Límites inferior y superior
    limite_inf = Q1 - multiplier * IQR
    limite_sup = Q3 + multiplier * IQR
    # Máscara booleana de outliers
    return (series < limite_inf) | (series > limite_sup)

In [ ]:
def detectar_outliers_zscore(series, umbral=3):
    # Cálculo de z-scores absolutos y máscara por umbral
    z_scores = np.abs(stats.zscore(series))
    return z_scores > umbral

### Limpieza específica basada en lógica de negocio

Copiamos del DataFrame para limpiar sin alterar el original

In [ ]:
df_cleaned = df.copy()
initial_rows = len(df_cleaned)

Regla: definimos precios medios > $500k como outliers

In [ ]:
outliers_target = df_cleaned['target'] > 5
print(f"Casas con precio > $500k: {outliers_target.sum()} registros")

Regla: promedio de habitaciones irreal (mayor que 30)

In [ ]:
outliers_rooms = df_cleaned['AveRooms'] > 30
print(f"Casas con >100 habitaciones promedio: {outliers_rooms.sum()} registros")

Regla: promedio de dormitorios irreal (mayor que 10)

In [ ]:
outliers_bedrms = df_cleaned['AveBedrms'] > 10
print(f"Casas con >10 dormitorios promedio: {outliers_bedrms.sum()} registros")

Regla: ocupación media muy alta > 50

In [ ]:
outliers_occup = df_cleaned['AveOccup'] > 50
print(f"Bloques con ocupación >50 personas/casa: {outliers_occup.sum()} registros")

Regla: población extremadamente alta por bloque (Population >7.500)

In [ ]:
outliers_pop = df_cleaned['Population'] > 7500
print(f"Bloques con población > 7.500: {outliers_pop.sum()} registros")

Regla: edad de la vivienda negativa o >100 años (HouseAge negativa o > 100 años)

In [ ]:
outliers_age = (df_cleaned['HouseAge'] < 0) | (df_cleaned['HouseAge'] > 100)
print(f"Casas con edad <0 o >100 años: {outliers_age.sum()} registros")

### Combinación de máscaras de outliers por reglas de negocio

In [ ]:
# Combinar todos los outliers lógicos
logical_outliers = (outliers_target | outliers_rooms | outliers_bedrms |
                   outliers_occup | outliers_pop | outliers_age)

In [ ]:
# Conteo de outliers por lógica de negocio
print(f"\nTotal de outliers por lógica de negocio: {logical_outliers.sum()}")

### Limpieza adicional con IQR

In [ ]:
# DataFrame auxiliar para marcar outliers por IQR
# Detectar outliers estadísticos en variables numéricas clave
iqr_outliers = pd.DataFrame(index=df_cleaned.index)

In [ ]:
# Marcar outliers por IQR para cada variable clave
for col in ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']:
    iqr_outliers[col] = detectar_outliers_iqr(df_cleaned[col])
    print(f"- {col}: {iqr_outliers[col].sum()} outliers detectados")

Marcar registros con >= x variables extremas

In [ ]:
# Considerar outlier si es extremo en múltiples variables
for i in range(4):
    multiple_outliers = (iqr_outliers.sum(axis=1) >= i)
    print(f'Registros con outliers en mas de {i} variables: {multiple_outliers.sum()}')

En nuestro caso, consideramos que con 2 o más variables atipicas es un outlier que debe eliminarse. Dependiendo de los datos que se eliminen, podríamos subir el umbral.

In [ ]:
multiple_outliers = (iqr_outliers.sum(axis=1) >= 1)
multiple_outliers.sum()

### Aplicamos la limpieza
Unimos outliers lógicos y estadísticos y filtramos

In [ ]:
all_outliers = logical_outliers | multiple_outliers
df_cleaned = df_cleaned[~all_outliers].reset_index(drop=True)

In [ ]:
print(f'df original: {len(df)} y df_cleaned {len(df_cleaned)}')

In [ ]:
print(f'Registros eliminados: {initial_rows - len(df_cleaned)}')
print(f'Porcentaje eliminado: {((initial_rows - len(df_cleaned)) / initial_rows * 100):.2f}%')
print(f'Dimensiones finales: {df_cleaned.shape}')

### Comparación de las estadísticas antes y después

In [ ]:
# Estadísticas del dataset original (variables seleccionadas)
df[['target', 'AveRooms', 'AveBedrms', 'AveOccup', 'Population']].describe()

In [ ]:
# Estadísticas del dataset limpio (mismas variables)
df_cleaned[['target', 'AveRooms', 'AveBedrms', 'AveOccup', 'Population']].describe()

### Visualización de outliers después de limpiar

In [ ]:
plot_outliers(df_cleaned, "Outliers Después de la Limpieza")

### Analizamos las correlaciones

In [ ]:
correlation_matrix = df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title("Matriz de Correlaciones - Dataset original")
plt.tight_layout()
plt.show()

En la siguiente matriz de correlaciones, se observa como ciertas variables han cambiado su correlación. Por ejemplo, el número de habitaciones ha pasado de un 15% a un 26%, y el promedio de ocupantes por vivienda en el área ha pasado a correlacionar de un -2.4% a un -28%.

In [ ]:
correlation_matrix = df_cleaned.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title("Matriz de Correlaciones - Dataset Limpio")
plt.tight_layout()
plt.show()

Guardamos dataset limpio en formato Feather, para utilizar en próximos ejemplos:

In [ ]:
df_cleaned.to_feather('cdiae/data/03_model_input/california_housing_clean.ftr')